In [2]:
import pandas as pd

In [4]:
data_path = "../datasets/fb15k-237/"

In [5]:
column_names = ['src_freebase_id', 'relation', 'dest_freebase_id']
triples_df = pd.read_csv(f"{data_path}/test.txt", header=None, names=column_names)

In [7]:
base_date = pd.Timestamp('2025-01-02')

num_triples = len(triples_df)
# random_days_ago_integers = np.random.randint(1, 90, size=num_triples)
# time_deltas = pd.to_timedelta(random_days_ago_integers, unit='days')

triples_df['timestamp'] = base_date

In [9]:
relations_df = pd.read_csv(f"{data_path}/derived/relations_with_ids.csv")

In [10]:
freebase_vs_wiki_df =  pd.read_csv(f"{data_path}/fb_wiki_mapping.tsv", sep='\t')

In [11]:
# strip Q from wikidata id for easier use with Kumo AI
freebase_vs_wiki_df['wikidata_int_id'] = freebase_vs_wiki_df.apply(lambda row: int(row['wikidata_id'][1:]), axis=1)
freebase_vs_wiki_df.drop('wikidata_id', axis=1, inplace=True)
freebase_vs_wiki_df.rename(columns={'wikidata_int_id':'wikidata_id'}, inplace=True)

In [13]:
triples_use_relation_id_df =  pd.merge(triples_df, relations_df, left_on ='relation', right_on='relation_name')

In [14]:
triplets_with_src_wiki_ids_df = pd.merge(triples_use_relation_id_df, freebase_vs_wiki_df, left_on='src_freebase_id', right_on='freebase_id')

In [15]:
triplets_with_src_dst_wiki_ids_df = pd.merge(triplets_with_src_wiki_ids_df, freebase_vs_wiki_df, left_on='dest_freebase_id', right_on='freebase_id')

In [16]:
triplets_with_src_dst_wiki_ids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18286 entries, 0 to 18285
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype        
---  ------            --------------  -----        
 0   src_freebase_id   18286 non-null  object       
 1   relation          18286 non-null  object       
 2   dest_freebase_id  18286 non-null  object       
 3   timestamp         18286 non-null  datetime64[s]
 4   relation_name     18286 non-null  object       
 5   relation_id       18286 non-null  int64        
 6   freebase_id_x     18286 non-null  object       
 7   label_x           18286 non-null  object       
 8   wikidata_id_x     18286 non-null  int64        
 9   freebase_id_y     18286 non-null  object       
 10  label_y           18286 non-null  object       
 11  wikidata_id_y     18286 non-null  int64        
dtypes: datetime64[s](1), int64(3), object(8)
memory usage: 1.7+ MB


In [17]:
triplets_with_src_dst_wiki_ids_df.rename(columns={'wikidata_id_x':'src_wikidata_id', 'wikidata_id_y':'dest_wikidata_id'}, inplace=True)

In [18]:
# get max id for train
train_triples_df = pd.read_csv(f"{data_path}/derived/fb_train_triples.csv")

In [20]:
train_triples_df.triple_id.describe()

count    272115.000000
mean     136057.000000
std       78552.978588
min           0.000000
25%       68028.500000
50%      136057.000000
75%      204085.500000
max      272114.000000
Name: triple_id, dtype: float64

In [23]:
max_triple_id_train = 272114
triplets_with_src_dst_wiki_ids_df['triple_id'] = triplets_with_src_dst_wiki_ids_df.index + max_triple_id_train + 1

In [24]:
triplets_with_src_dst_wiki_ids_df.to_csv(f"{data_path}/derived/triplets_with_src_dst_wiki_ids.test.csv", index=False)